In [ ]:
!pip install roboticstoolbox-python
!pip install ipympl

In [13]:
import roboticstoolbox as rtb
import spatialmath as sm
from math import pi, sin, cos
import numpy as np
import matplotlib.pyplot as plt

In [14]:
%matplotlib widget

In [15]:
robot = rtb.DHRobot( 
    [ 
        rtb.RevoluteDH(a=1), 
        rtb.RevoluteDH(a=1) 
    ], 
    name="two_link")
print(robot)


┏━━━━┳━━━━━┳━━━━┳━━━━━━┓
┃θⱼ  ┃ dⱼ  ┃ aⱼ ┃  ⍺ⱼ  ┃
┣━━━━╋━━━━━╋━━━━╋━━━━━━┫
┃ q1 ┃ 0.0 ┃  1 ┃ 0.0° ┃
┃ q2 ┃ 0.0 ┃  1 ┃ 0.0° ┃
┗━━━━┻━━━━━┻━━━━┻━━━━━━┛



Para nuestro ejemplo l1 y l2 miden 1 
De acuerdo con la cinemática directa para un brazo planar
$$ x = \cos(q1) + \cos(q1+q2) $$
$$ y = \sin(q1) + \sin(q1+q2) $$
$$ \begin{eqnarray} J_0 & = & \left [ \begin{array}{cc}\frac{\partial x}{\partial q1} && \frac{\partial x}{\partial q2} \\ \frac{\partial y}{\partial q1} && \frac{\partial y}{\partial q2}  \end{array} \right ]\\ & = & \left [ \begin{array}{cc}-\sin(q1) -\sin(q1+q2) && -\sin(q1+q2) \\ \cos(q1) + \cos(q1+q2) && \cos(q1+q2)  \end{array}\right ] \end{eqnarray}$$

 End-effector spatial velocity $$\nu = (v_x, v_y, v_z, \omega_x, \omega_y, \omega_z)^T$$
        is related to joint velocity by $${}^{0}\!\nu = \mathbf{J}_0(q) \dot{q}$$

robot.jacob_dot(q, qd) computes the product (6) of the temporal
derivative of the manipulator Jacobian (in the world frame) and the
joint rates $\frac{d}{dt} \dot{\mathbf{J}}(q) \dot{q}$.
This term appears in the formulation for operational space control $\ddot{x} = \mathbf{J}(q) \ddot{q} + \dot{\mathbf{J}}(q) \dot{q}$.
            


In [23]:
q = np.array([0,pi])
p = np.array([cos(q[0]) + cos(q[0]+q[1]),sin(q[0]) + sin(q[0]+q[1])])
q_punto = np.array([.00017, .00034]) # 1 grado y 2 grados
q_star = q + q_punto
p_punto = np.dot(jacobian(q), q_punto)
print('J=',jacobian (q))
print('p=',p)
print('p_punto',p_punto)
p_star = p[0:2] + p_punto
print('p_star',p_star)
print('directa(q_star)',robot.fkine(q_star).t)

J= [[-1.2246468e-16 -1.2246468e-16]
 [ 0.0000000e+00 -1.0000000e+00]]
p= [0.0000000e+00 1.2246468e-16]
p_punto [-6.24569868e-20 -3.40000000e-04]
p_star [-6.24569868e-20 -3.40000000e-04]
directa(q_star) [ 1.15599997e-07 -3.39999979e-04  0.00000000e+00]


In [ ]:
q = np.array([pi/4,pi/6])
T = robot.fkine(q)
p = T.t
print(q)
print(p)
print(T.rpy(unit='deg'))
robot.plot(q)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
q_punto = np.array([.017, .034]) # 1 grado y 2 grados
q_star = q + q_punto
print(q_star)

In [16]:
def jacobian(q):
    J = np.array([ [-sin(q[0]) + -sin(q[0]+q[1]), -sin(q[0]+q[1])],
                   [cos(q[0]) + cos(q[0]+q[1]), cos(q[0]+q[1])] ])
    return J

In [ ]:
p_punto = np.dot(jacobian(q), q_punto)
print(jacobian (q))
print(p)
print(p_punto)
p_star = p[0:2] + p_punto
print(p_star)
print(robot.fkine(q_star).t)

In [ ]:
robot.plot(q_star)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
T = sm.SE3(p_star[0],p_star[1],0)
q_prima, *_ = robot.ikine(T,mask=[1,1,0,0,0,0])
print(q_prima)
robot.plot(q_prima)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
## print(robot.jacobe(q)) # Jacobian in end-effector frame
print(robot.jacob0(q))# Jacobian in world frame
#print(robot.jacob_dot(q,q_punto))# Jacobe dot in end-effector frame

In [ ]:
qt = rtb.tools.trajectory.jtraj(q, q_star, 50)
qt.q

In [ ]:
robot.plot(qt.q, dt=0.1);